In [1]:

from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F

In [1]:
from torchvision.models import efficientnet_b0

In [ ]:
# read Temperature from the HDF5 file and store it in the notebook's `data` dict
with h5py.File(hdf5_path, 'r') as fh:
    # adjust the run name if needed (here it's 'run1' per earlier cells)
    run = 'run1'
    if run not in fh:
        raise KeyError(f"Run '{run}' not found in {hdf5_path}. Available: {list(fh.keys())}")
    if key not in fh[run]:
        raise KeyError(f"Dataset '{key}' not found in {run}. Available: {list(fh[run].keys())}")
    temp = fh[run][key][:]
    data[key] = temp

# inspect
print(key, "->", type(data[key]), "shape:", getattr(data[key], "shape", None), "dtype:", getattr(data[key], "dtype", None))

In [90]:
import h5py
import numpy as np
import torch

hdf5_path = '/raid/javier/Projects/PhaseClassifier/Dataset/data00.h5'
data = {'Temperature': [], 'Snapshot': []}
with h5py.File(hdf5_path, 'r') as f:
    print(f.keys())
    print(f['run1'].keys())
    for key1 in f.keys():
        for key in f[key1].keys():
            # print(f[key1][key])
            if key == 'Temperature':
                tmp = f[key1][key]
                data['Temperature'].append(tmp[()])
            if key == 'Snapshot':
                tmp = f[key1][key]
                data['Snapshot'].extend(torch.tensor(tmp[()]).unsqueeze(0))  # add channel dim


<KeysViewHDF5 ['run0', 'run1', 'run10', 'run100', 'run1000', 'run1001', 'run1002', 'run1003', 'run1004', 'run1005', 'run1006', 'run1007', 'run1008', 'run1009', 'run101', 'run1010', 'run1011', 'run1012', 'run1013', 'run1014', 'run1015', 'run1016', 'run1017', 'run1018', 'run1019', 'run102', 'run1020', 'run1021', 'run1022', 'run1023', 'run1024', 'run1025', 'run1026', 'run1027', 'run1028', 'run1029', 'run103', 'run1030', 'run1031', 'run1032', 'run1033', 'run1034', 'run1035', 'run1036', 'run1037', 'run1038', 'run1039', 'run104', 'run1040', 'run1041', 'run1042', 'run1043', 'run1044', 'run1045', 'run1046', 'run1047', 'run1048', 'run1049', 'run105', 'run1050', 'run1051', 'run1052', 'run1053', 'run1054', 'run1055', 'run1056', 'run1057', 'run1058', 'run1059', 'run106', 'run1060', 'run1061', 'run1062', 'run1063', 'run1064', 'run1065', 'run1066', 'run1067', 'run1068', 'run1069', 'run107', 'run1070', 'run1071', 'run1072', 'run1073', 'run1074', 'run1075', 'run1076', 'run1077', 'run1078', 'run1079', 

In [91]:
torch.tensor(data['Temperature'])

tensor([2.8000, 2.8000, 2.8000,  ..., 2.8000, 2.8000, 2.8000],
       dtype=torch.float64)

In [82]:
torch.cat(data['Snapshot'],dim=0).shape

torch.Size([160000, 32])

In [93]:
# data['Snapshot']
32*32

1024